In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from numpy.linalg import inv
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from numpy import expand_dims
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from matplotlib.colors import ListedColormap
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
def image_array(name):
    img = load_img(name)
    img_array = img_to_array(img)
    return img_array

In [ ]:
def block_img(img_name):
  img = image_array(img_name)
  h = 8
  w = 8
  feature = []
  for a in range(0, (img.shape[0]-h)+1, h):
    for b in range(0, (img.shape[1]-w)+1, w):
      block = img[a:a+h, b:b+w]
      my_block = (block.flatten())
      feature.append(my_block)
  return feature

In [ ]:
cat1, cat2, dog1, dog2 = block_img('cat1.jpg'), block_img('cat2.jpg'),\
      block_img('dog1.jpg'), block_img('dog2.jpg')

In [ ]:
df1, df2, df3, df4 = pd.DataFrame(cat1), pd.DataFrame(cat2),\
pd.DataFrame(dog1), pd.DataFrame(dog2)

In [ ]:
df1['label'], df2['label'] =0,0
df3['label'], df4['label'] = 1,1

In [ ]:
df = pd.concat([df1, df2, df3, df4], axis =0)
df.head()

In [ ]:
#Plotting 2D for ds
x = df[7]
y = df[35]
label = df["label"]
color = ['orange', 'black']
fig = plt.figure(figsize=(6,4))
plt.scatter(x, y, c=label, cmap=matplotlib.colors.ListedColormap(color))
plt.title("2D")
plt.xlabel("7th feature")
plt.ylabel("35th feature")

In [ ]:
def checker(df):
    if df.isnull().sum().sum()==0:
        print("Full data")
    if df.shape[1]<df.shape[0]:
        print("Low dimensional")
    a = df.describe()
    if np.max(a[a.index=='mean'].values)-np.min(a[a.index=='mean'].values)>100:
        print("Scale problem")
checker(df)

In [ ]:
f3 = [df[7]]
f7 = [df[9]]
plt.hist(f3, alpha=0.5, label='f3', bins=30)
plt.hist(f7, alpha=0.5, label='f7', bins=30)
plt.legend(loc='upper right')
plt.show()

In [ ]:
import plotly.graph_objects as go

# Data
f3 = df[7]
f7 = df[9]

# Create histogram traces
trace_f3 = go.Histogram(
    x=f3,
    name='f3',
    opacity=0.5,
    nbinsx=30
)
trace_f7 = go.Histogram(
    x=f7,
    name='f7',
    opacity=0.5,
    nbinsx=30
)

# Create layout
layout = go.Layout(
    title='Histogram of f3 and f7',
    xaxis=dict(title='Values'),
    yaxis=dict(title='Frequency'),
    barmode='overlay'
)

# Create figure
fig = go.Figure(data=[trace_f3, trace_f7], layout=layout)

# Display the figure
fig.show()

In [ ]:
df.boxplot(column = [7,8,9,10,11,12])
df.describe().T

In [ ]:
import plotly.graph_objects as go

# Select the desired columns
columns = [7, 8, 9, 10, 11, 12]

# Create box plot traces
traces = []
for column in columns:
    trace = go.Box(
        y=df[column],
        name=f'Column {column}',
        boxpoints='all',
        jitter=0.5,
        whiskerwidth=0.2,
        marker=dict(size=2),
        line=dict(width=1)
    )
    traces.append(trace)

# Create layout
layout = go.Layout(
    title='Box Plot',
    yaxis=dict(title='Values')
)

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Display the figure
fig.show()

In [ ]:
def scaling(df):
    X = df.iloc[:, 0:192].values
    y = df.iloc[:, 192].values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=0, shuffle = True)
        
    my_scaler = preprocessing.StandardScaler()

    X_train_std = my_scaler.fit_transform(X_train)
    X_test_std = my_scaler.fit_transform(X_test)
    return X_train_std, X_test_std,  y_train, y_test

In [ ]:
X_train_std, X_test_std,  y_train, y_test = scaling(df)

In [ ]:
def lr_lda(df):
    X_train_std, X_test_std,  y_train, y_test = scaling(df)
    lr_normal = LogisticRegression()
    lr_normal = lr_normal.fit(X_train_std, y_train)
    pred_normal=lr_normal.predict(X_test_std)
    print ("Classification Report")
    print(classification_report(y_test, pred_normal))
    print ("Confusion Report")
    print(confusion_matrix(y_test, pred_normal))
    return lr_normal, pred_normal

In [ ]:
lr_normal, pred_normal = lr_lda(df)

In [ ]:
sns.displot(X_train_std[3], kde = True)
plt.legend(loc='upper right')
plt.show()

In [ ]:
covMatrix_train = np.cov(y_train,bias=True)
covMatrix_test = np.cov(X_test_std, bias = True)
print(covMatrix_train)

In [ ]:
X_train_std, X_test_std,  y_train, y_test = scaling(df)
X_train_std

In [ ]:
def LDA(df):
    X_train_std, X_test_std,  y_train, y_test = scaling(df)
    n_components = min(X_train_std.shape[1], len(np.unique(y_train))) - 1
    lda = LinearDiscriminantAnalysis(n_components=n_components)
    X_train_LDA = lda.fit_transform(X_train_std, y_train)
    X_test_LDA = lda.fit_transform(X_test_std, y_test)

    lr_lda = LogisticRegression()
    lr_lda = lr_lda.fit(X_train_LDA, y_train)

    pred_lda=lr_lda.predict(X_test_LDA)
    print ("Classification Report")
    print(classification_report(y_test, pred_lda))
    print ("Confusion Report")
    print(confusion_matrix(y_test, pred_lda))
    return pred_lda

In [ ]:
pred_lda = LDA(df)

In [ ]:
def LogisticRegression_PCA(df):
    X_train_std, X_test_std,  y_train, y_test = scaling(df)

    pca = PCA(n_components= 2) 
    X_train_PCA = pca.fit_transform(X_train_std)
    X_test_PCA = pca.transform(X_test_std)

    log_r1 = LogisticRegression()
    log_r1.fit(X_train_PCA, y_train)

    pred1=log_r1.predict(X_test_PCA)

    print ("Classification Report")
    print(classification_report(y_test, pred1))

    print ("Confusion Report")
    print(confusion_matrix(y_test, pred1))

    print(pca.explained_variance_ratio_)
    return pred1

In [ ]:
pred1 = LogisticRegression_PCA(df)

In [ ]:
def PCA_LDA_Random(df):
    X_train_std, X_test_std,  y_train, y_test = scaling(df)

    pipeline = Pipeline([('pca', PCA(n_components= 2)), ('clf',  RandomForestRegressor(n_estimators=20,random_state=42))])
    pipeline.fit(X_train_std, y_train)
    print('Test Accuracy: %.3f' %pipeline.score(X_test_std, y_test))

    n_components = min(X_train_std.shape[1], len(np.unique(y_train))) - 1
    
    pipeline2 = Pipeline([('pca', LinearDiscriminantAnalysis(n_components= n_components)), \
                         ('clf', RandomForestRegressor(n_estimators = 20, random_state=42))])

    pipeline2.fit(X_train_std, y_train)
    print('Test Accuracy: %.3f' %pipeline2.score(X_test_std, y_test))
    

In [ ]:
PCA_LDA_Random(df)

In [ ]:


def Sequential_Model(df):
    X_train_std, X_test_std, y_train, y_test = scaling(df)

    classifier = Sequential()
    classifier.add(Dense(units=800, kernel_initializer='uniform', activation='relu', input_shape=(192,)))
    classifier.add(Dense(units=600, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=400, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=2, kernel_initializer='uniform', activation='sigmoid'))

    optimizer = tf.keras.optimizers.SGD()
    classifier.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])
    classifier.fit(X_train_std, y_train, epochs=10, batch_size=32)

    predictions = classifier.predict(X_test_std)
    prediction = tf.argmax(predictions, axis=1)

    CC_test = tf.math.confusion_matrix(y_test, prediction)
    TN = CC_test[1, 1]
    FP = CC_test[1, 0]
    FN = CC_test[0, 1]
    TP = CC_test[0, 0]
    FPFN = FP + FN
    TPTN = TP + TN
    Accuracy = 1 / (1 + (FPFN / TPTN))
    print("Our_Accuracy_Score:", Accuracy)
    Precision = 1 / (1 + (FP / TP))
    print("Our_Precision_Score:", Precision)
    Sensitivity = 1 / (1 + (FN / TP))
    print("Our_Sensitivity_Score:", Sensitivity)
    Specificity = 1 / (1 + (FP / TN))
    print("Our_Specificity_Score:", Specificity)

    return prediction


In [ ]:
Sequential_Model(df)